**Requirements**:

* QMCPy: `pip install qmcpy==2.1`
* LaTeX: `sudo apt update && sudo apt install -y texlive-full`
* testbook : `pip install testbook==0.4.2`
* Parsl: `pip install parsl==2025.7.28`

This notebook can be run interactively or in command line mode. To run in command line mode, use:
```bash
    jupyter nbconvert --to notebook --execute demos/talk_paper_demos/parsel_fest_2025/parsl_fest_2025.ipynb \
  --ExecutePreprocessor.kernel_name=qmcpy --ExecutePreprocessor.timeout=3600 --inplace
```

Our presentation slides for ParslFest are available at [Figma](https://www.figma.com/slides/k7EUosssNluMihkYTLuh1F/Parsl-Testbook-Speedup?node-id=174-95&t=t3jENVMltXWwdLdb-0).

In [1]:
try:
    import parsl as pl
except ModuleNotFoundError:
    !pip install -q parsl

In [2]:
import sys
import os
import time
import subprocess

# Ensure the path to the booktests directory is included (robust finder)
def _find_repo_root(start=os.getcwd()):
    cur = start
    while True:
        if os.path.exists(os.path.join(cur, 'pyproject.toml')):
            return cur
        parent = os.path.dirname(cur)
        if parent == cur:
            raise FileNotFoundError('repo root not found')
        cur = parent

sys.path.append(os.path.join(_find_repo_root(), 'test', 'booktests'))

# Configuration flags: set force_compute=False to reuse existing outputs; set True to force re-run
force_compute = True
is_debug = True 

# Create output directory if it doesn't exist
print(f"{os.getcwd() = }")
output_dir = "output"
os.makedirs(output_dir, exist_ok=True)

# Clean local only files if force_compute is set
if force_compute:
    !(cd ../../.. && make clean_local_only_files)
    # remove output directory contents
    !rm -fr output/*.* runinfo/

os.getcwd() = '/Users/terrya/Documents/ProgramData/QMCSoftware_resume/demos/talk_paper_demos/parsel_fest_2025'
rm -fr test/booktests/.ipynb_checkpoints/
chmod +x scripts/find_local_only_folders.sh > /dev/null 2>&1
for f in ; do \
		rm -f "$f"; > /dev/null 2>&1; \
	done
rm -fr test/booktests/.ipynb_checkpoints/
chmod +x scripts/find_local_only_folders.sh > /dev/null 2>&1
for f in ; do \
		rm -f "$f"; > /dev/null 2>&1; \
	done


## 1. Sequential Execution

In [3]:
seq_fname = os.path.join(output_dir, "sequential_time.csv")
is_linux = sys.platform.startswith("linux")
if (not os.path.exists(seq_fname)) or force_compute:  # build the make command
    if is_debug:
        tests = "tb_quickstart tb_qmcpy_intro"
        cmd = ["make", "booktests_no_docker", f"TESTS={tests}"]
    else:
        cmd = ["make", "booktests_no_docker"]
    if is_linux:
        cmd = ["taskset", "-c", "0"] + cmd
    out_path = os.path.join(output_dir, "sequential_output.csv")
    repo_root = _find_repo_root()
    start_time = time.time()
    with open(out_path, 'wb') as out_f:
        try:
            subprocess.run(cmd, cwd=repo_root, stdout=out_f, stderr=subprocess.STDOUT, check=True)
        except subprocess.CalledProcessError:
            pass
    sequential_time = time.time() - start_time
 
    with open(seq_fname, "w") as f:    # save sequential time to a file in output_dir
        _ = f.write(f"{sequential_time:.2f}\n")

CompletedProcess(args=['make', 'booktests_no_docker', 'TESTS=tb_quickstart tb_qmcpy_intro'], returncode=0)

In [4]:
# free memory
import gc
gc.collect();